In [6]:
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF

# Carrega o modelo de embeddings
modelo = SentenceTransformer('all-MiniLM-L6-v2')  # Você pode escolher outro modelo Sentence-BERT

def gerar_embeddings_paginas(pdf_path):
    """
    Gera embeddings para cada página do PDF e armazena em um dicionário.
    """
    embeddings_paginas = {}
    documento = fitz.open(pdf_path)
    
    for numero_pagina in range(documento.page_count):
        pagina = documento.load_page(numero_pagina)
        texto_pagina = pagina.get_text()  # Extrai o texto da página
        
        # Gera o embedding da página
        embedding = modelo.encode(texto_pagina, convert_to_tensor=True)
        embeddings_paginas[numero_pagina + 1] = embedding
    
    return embeddings_paginas

def encontrar_paginas_relevantes(questao, embeddings_paginas, top_n=5):
    """
    Dada uma questão, retorna as páginas mais relevantes usando embeddings de similaridade.
    """
    # Gera o embedding da questão
    embedding_questao = modelo.encode(questao, convert_to_tensor=True)
    
    # Calcula a similaridade entre o embedding da questão e cada embedding de página
    similaridades = []
    for numero_pagina, embedding_pagina in embeddings_paginas.items():
        similaridade = util.pytorch_cos_sim(embedding_questao, embedding_pagina).item()
        similaridades.append((numero_pagina, similaridade))
    
    # Ordena as páginas pela similaridade e seleciona as top_n páginas mais relevantes
    paginas_relevantes = sorted(similaridades, key=lambda x: x[1], reverse=True)[:top_n]
    return paginas_relevantes

# Exemplo de uso
pdf_path = 'humanas.pdf'  # Substitua pelo caminho do seu PDF
embeddings_paginas = gerar_embeddings_paginas(pdf_path)

from nltk.corpus import stopwords

pt_stp_words = stopwords.words('portuguese')

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word.lower() not in pt_stp_words])

# Exemplo de questão
questao = 'Uma das características a ser reconhecida no feudalismo europeu é: A sociedade feudal era semelhante ao sistema de castas. Os ideais de honra e fidelidade vieram das instituições dos hunos. Vilões e servos estavam presos a várias obrigações, entre elas, o pagamento anual de capitação, talha e banalidades. A economia do feudo era dinâmica, estando voltada para o comércio dos feudos vizinhos. As relações de produção eram escravocratas.'
questao = remove_stopwords(questao)

# Busca as páginas mais relevantes
paginas_relacionadas = encontrar_paginas_relevantes(questao, embeddings_paginas, top_n=5)

print("Páginas mais relevantes para a questão:")
for pagina, similaridade in paginas_relacionadas:
    print(f"Página {pagina} com similaridade {similaridade:.4f}")

Páginas mais relevantes para a questão:
Página 119 com similaridade 0.7045
Página 17 com similaridade 0.6665
Página 118 com similaridade 0.6586
Página 148 com similaridade 0.6175
Página 175 com similaridade 0.6096
